잠재의미분석 (Latent Semantic Analysis LSA)

##### 1) 직접 구현

In [6]:
#CountVectorizer를 사용하여 텍스트 데이터 -> 단어 빈도 행렬(DTM, Document-Term Matrix)

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer() #CountVectorizer 객체 생성 (텍스트를 처리하여 단어의 빈도 계산)
DTM = cv.fit_transform(docs).toarray() #to_array() 메소드를 사용하여 희소 행렬 -> 밀집 형태로
feature_name = cv.get_feature_names_out() #DTM의 열(피처)에 해당하는 단어 목록
word2id = cv.vocabulary_  # 딕셔너리도 따로 선헌해둔다.

In [7]:
DTM

#DTM(Document-Term Matrix): 문서와 단어의 관계를 표현하는 행렬. 단어가 무엇인지는 확인하기 어려움

array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 2],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [8]:
cv.get_feature_names_out() #DTM의 각 열에 해당하는 단어들 확인

array(['가츠동', '김치', '김치찌개', '된장', '된장찌개', '라면', '바나나', '볶음밥', '비빔밥', '사과',
       '소바', '스시', '짜장면', '짬뽕', '탕수육', '포도'], dtype=object)

In [10]:
#randomized_svd를 사용하여 특이값 분해(Singular Value Decomposition, SVD) 수행 -> 문서-단어행렬(DTM)을 주어진 차원(K)으로 차원 축소
#각 주제에 속하는 상위 3개의 단어 출력. 각 주제의 주요 키워드 확인

from sklearn.decomposition import randomized_svd #특이값 분해(SVD)의 무작위 버전 제공

U, s, VT = randomized_svd(DTM, n_components = k, n_iter=10, random_state = 0) #randomized_svd 사용하여 DTM을 주워진 차원(K)로 차원 축소

for topic in VT : #축소된 행렬 VT의 각 행=주제(topic). VT의 각 행에 대해 반복
  print([feature_name[i] for i in topic.argsort()[::-1][:3]])

['포도', '바나나', '짜장면']
['짜장면', '짬뽕', '김치']
['김치', '된장찌개', '김치찌개']
['스시', '김치', '가츠동']


##### 2) sklearn 활용

In [13]:
doc_ls = ['바나나 사과 포도 포도 ',
         '사과 포도',
         '포도 바나나',
         '짜장면 짬뽕 탕수육',
         '볶음밥 탕수육',
         '짜장면 짬뽕',
         '라면 스시',
         '스시 ',
         '가츠동 스시 소바',
         '된장찌개 김치찌개 김치',
         '김치 된장 ',
         '비빔밥 김치'
         ]


In [14]:
#TD-IDF를 사용하여 문서를 벡터화 -> Truncated SVD를 사용하여 주어진 토픽의 수로 차원 축소

from sklearn.feature_extraction.text import TfidfVectorizer #TF-IDF 벡터화를 위한 TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

n_topics = 4 #토픽의 수=4

tfidfv = TfidfVectorizer() #TfidfVectorizer 객체 생성. TF-IDF 가충지를 사용하여 단어를 벡터화
tfidf = tfidfv.fit_transform(docs) #TF-IDF 벡터화. docs=입력된 문서들
svd = TruncatedSVD(n_components = n_topics, algorithm = 'randomized', n_iter=100) #n_components: 주어진 토픽의 수
svd.fit_transform(tfidf) #문서집합을 토픽의 수로 차원 축소

array([[ 0.73533636,  0.65297083,  0.10110344,  0.01060989],
       [ 0.61363093,  0.60431072,  0.10524359,  0.01238742],
       [ 0.7984386 ,  0.26045607, -0.01831414, -0.00672613],
       [ 0.53305177, -0.56031022, -0.42213385, -0.33044636],
       [ 0.14418228, -0.21254471, -0.23191093, -0.2625725 ],
       [ 0.54801361, -0.53006465, -0.34992723, -0.22258649],
       [ 0.406527  , -0.48568088,  0.49477573,  0.00250927],
       [ 0.18446514, -0.3626629 ,  0.77698444,  0.02307865],
       [ 0.12607679, -0.273366  ,  0.66103424,  0.02235352],
       [ 0.07786141, -0.11341968, -0.1305914 ,  0.67529106],
       [ 0.35852776, -0.33318007, -0.21842001,  0.55386528],
       [ 0.09205415, -0.13080845, -0.1456878 ,  0.72457634]])

In [16]:
#Truncated SVD를 통해 추출된 첫 번째 주제의 주요 단어들을 출력 => 주요 주제 파악

[feature_name[i] for i in svd.components_[0].argsort()[::-1][:3]]

['포도', '짜장면', '바나나']

In [17]:
#Truncated SVD를 통해 추출된 각 주제의 상위 3개의 주요 단어를 출력

for idx, topic in enumerate(svd.components_) :
  print([feature_name[i] for i in topic.argsort()[::-1][:3]])

#svd.components_: Truncated SVD를 통해 추출된 주성분들을 담고 있는 배열
#enumerate(): 주성분들의 인덱스와 해당 주성분 순회
#topic.argsort(): 각 주제의 주성분을 구성하는 값들을 정렬하여 그에 해당하는 인덱스들 반환 => 단어의 중요도

['포도', '짜장면', '바나나']
['포도', '사과', '바나나']
['스시', '소바', '가츠동']
['김치', '비빔밥', '된장찌개']


##### 3) gensim 활용

In [18]:
docs = ['바나나 사과 포도 포도',
         '사과 포도',
         '포도 바나나',
         '짜장면 짬뽕 탕수욕',
         '볶음밥 탕수욕',
         '짜장면 짬뽕',
         '라면 스시',
         '스시',
         '가츠동 스시 소바',
         '된장찌개 김치찌개 김치',
         '김치 된장',
         '비빔밥 김치'
         ]

In [19]:
#'docs'라는 리스트 안에 있는 각 문서들을 공백을 기준으로 분리하여 토큰화한 결과를 리스트 'docs_ls'에 저장

doc_ls = [doc.split() for doc in docs]
doc_ls[0] #첫 번째 문서를 토큰화한 결과

['바나나', '사과', '포도', '포도']

In [20]:
#Latent Semantic Analysis (LSA) 수행 -> 4개의 주제를 가진 LSI 모델 생성

from gensim import corpora
from gensim.models import LsiModel
from gensim.models import TfidfModel

id2word = corpora.Dictionary(doc_ls)
corpus_TDM = [id2word.doc2bow(t) for t in doc_ls]
model_LSA = LsiModel(corpus_TDM, id2word=id2word, num_topics = 4)

In [21]:
model_LSA.print_topics(4, 3)

[(0, '0.816*"포도" + 0.408*"사과" + 0.408*"바나나"'),
 (1, '-0.612*"짜장면" + -0.612*"짬뽕" + -0.484*"탕수욕"'),
 (2, '-0.813*"김치" + -0.337*"된장찌개" + -0.337*"김치찌개"'),
 (3, '-0.815*"스시" + -0.368*"가츠동" + -0.368*"소바"')]